# In-Depth Guide: Prediction

This notebook covers how to use a trained **detectree2** model to make predictions on large-scale orthomosaics, including tiling for prediction, running inference, and the full postprocessing pipeline (`stitch`, `clean`, `post_clean`).

For the full tutorial, see the [documentation](https://patball1.github.io/detectree2/tutorials/04_prediction.html).

## Setup

In [ ]:
!pip install torch torchvision torchaudio
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install detectree2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Tiling the Orthomosaic for Prediction

Tile the entire orthomosaic so that a crown map can be made for the full landscape. Tiles should be approximately the same size as those trained on (~100 m). A buffer (here 30 m) is included so we can discard partial crowns at tile edges.

In [ ]:
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns, post_clean
from detectree2.models.predict import predict_on_data
from detectree2.models.train import setup_cfg
from detectron2.engine import DefaultPredictor
import rasterio

In [ ]:
# Path to site folder and orthomosaic
site_path = "/path/to/data/BCI_50ha"
img_path = site_path + "/rgb/2015.06.10_07cm_ORTHO.tif"
tiles_path = site_path + "/tilespred/"

# Specify tiling
buffer = 30
tile_width = 40
tile_height = 40
tile_data(img_path, tiles_path, buffer, tile_width, tile_height, dtype_bool=True)

> **Warning:** If tiles are outputting as blank images, set `dtype_bool=True`. Avoid supplying crown polygons otherwise the function will run as if tiling for training.

### Downloading a Pre-trained Model

You can download a pre-trained model from the model garden:

In [ ]:
!wget https://zenodo.org/records/15863800/files/250312_flexi.pth

## 2. Running Predictions

In [ ]:
trained_model = "./230103_randresize_full.pth"
cfg = setup_cfg(update_model=trained_model)
predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))

## 3. Postprocessing

Project predictions back into geographic space, then stitch and clean.

In [ ]:
# Project tile predictions to geo-referenced crowns
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")

In [ ]:
# Stitch crowns together, handling overlaps in the buffer
crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)

# Clean overlapping predictions (removes lower-confidence duplicates)
clean = clean_crowns(crowns, 0.6, confidence=0)  # set confidence>0 to filter less confident crowns

Optionally filter by confidence score. `Confidence_score` is a column in the crowns GeoDataFrame and is a tunable parameter.

In [ ]:
clean = clean[clean["Confidence_score"] > 0.5]

## 4. Filling Gaps with `post_clean`

`clean_crowns` aggressively removes overlapping predictions, which can leave gaps. `post_clean` iteratively fills these gaps by adding back crowns from the original (unclean) set that do not significantly overlap with the cleaned crowns. It uses a lower IoU threshold (default 0.3) and repeats until no new crowns are added.

In [ ]:
clean = post_clean(crowns, clean, iou_threshold=0.3)

You can control the maximum number of iterations (default 5):

In [ ]:
# Alternative with explicit max iterations
# clean = post_clean(crowns, clean, iou_threshold=0.3, max_iterations=3)

## 5. Simplify and Save

Crown polygons have many vertices (generated from pixelwise masks). Simplify for easier editing in GIS software. The `tolerance` has the same units as the CRS (meters for UTM).

In [ ]:
clean = clean.set_geometry(clean.simplify(0.3))

# Save the crown map
clean.to_file(site_path + "/crowns_out.gpkg")

You can now view `crowns_out.gpkg` in QGIS or ArcGIS.